In [2]:

import numpy as np
import cv2
import os
import sys
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn.metrics import f1_score



In [5]:
# Hyperparameters and settings
batch_size = 64
test_batch_size = 64
input_size = 1 # because there is only one channel 
output_size = 4
num_epochs = 1000
learning_rate = 0.001



In [6]:
# Load traiing, validation and training data

data_loader = torch.load('data_loader.pt')
valid_loader = torch.load('valid_loader.pt')
test_loader = torch.load('test_loader.pt')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the data loaders to the desired device
data_loader.dataset.tensors = tuple(tensor.to(device) for tensor in data_loader.dataset.tensors)
valid_loader.dataset.tensors = tuple(tensor.to(device) for tensor in valid_loader.dataset.tensors)
test_loader.dataset.tensors = tuple(tensor.to(device) for tensor in test_loader.dataset.tensors)


In [13]:
class CNN(nn.Module):
    def __init__(self):
        self.name = "CNN"
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(12 * 12 * 64, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        # conv layers
        x = self.conv_layer(x)
        # print(x.shape)
        # flatten
        x = x.view(x.size(0), -1)
        # print(x.shape)
        # fc layer
        x = self.fc_layer(x)
        return x



In [14]:
class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.name = "CNN2"
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            #get dimensions of last layer
            
        )
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(64*9*9, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        # conv layers
        x = self.conv_layer(x)
        # print(x.shape)
        # flatten
        x = x.view(x.size(0), -1)
        # print(x.shape)
        # fc layer
        x = self.fc_layer(x)
        return x

In [15]:
class CNN3(nn.Module):
    def __init__(self):
        super(CNN3, self).__init__()
        self.name = "CNN3"
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=7, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=7, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
           
            
        )
        
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(20*20*64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        # conv layers
        x = self.conv_layer(x)
        # print(x.shape)
        # flatten
        x = x.view(x.size(0), -1)
        # print(x.shape)
        # fc layer
        x = self.fc_layer(x)
        return x

In [26]:
class CNN4(nn.Module):
    def __init__(self):
        super(CNN4, self).__init__()
        self.name = "CNN4"
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),           
        )
        
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(12 * 12 * 64, 1000),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(1000, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(512, 10)
        )
        
    def forward(self, x, y=None):
        # conv layers
        x = self.conv_layer(x)
        # print(x.shape)
        # flatten
        x = x.view(x.size(0), -1)
        # print(x.shape)
        # fc layer
        x = self.fc_layer(x)
        return x

In [8]:
def train_and_save_model(model, data_loader, valid_loader, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    total_step = len(data_loader)
    loss_list = []
    acc_list = []
    f1_list = []

    loss_list_test = []
    acc_list_test = []
    f1_list_test = []

    # early stopping parameters
    best_loss = 100
    patience = 10 # number of epochs to wait before stopping
    count = 0
    best_epoch = 0
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(data_loader):

            # Move images and labels to the device
            images = images.to(device)
            labels = labels.to(device)
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss_list.append(loss.item())
            # Backprop and optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # Train accuracy
            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            acc_list.append(correct / total)
            # Train F1 score
            f1 = f1_score(labels.cpu().numpy(), predicted.cpu().numpy(), average='macro')
            f1_list.append(f1)
            # print(i)
            # if (i + 1) % 10 == 0:
            #     print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
            #     .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
            #     (correct / total) * 100))

        with torch.no_grad():
            for i, (images_test, labels_test) in enumerate(valid_loader):
                images_test = images_test.to(device)
                labels_test = labels_test.to(device)
                outputs_test = model(images_test)
                loss_test = criterion(outputs_test, labels_test)
                loss_list_test.append(loss_test.item())
                total_test = labels_test.size(0)
                _, predicted_test = torch.max(outputs_test.data, 1)
                correct_test = (predicted_test == labels_test).sum().item()
                acc_list_test.append(correct_test / total_test)

                #Test F1 score
                f1_test = f1_score(labels_test.cpu().numpy(), predicted_test.cpu().numpy(), average='macro')
                f1_list_test.append(f1_test)
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%, F1 Score: {:.2f}'
                .format(epoch + 1, num_epochs, i + 1, total_step, loss_test.item(),
                (correct / total) * 100, f1_test))
        
        print(loss_test.item())

        #================= Early stopping =================
        
        #---------- Early stopping with avg F1 score-------
        # early stopping to prevent overfitting
        # saving model with best accuracy
        # avg_f1 = sum(f1_list_test)/len(f1_list_test)
        # print('Average F1 score: ', avg_f1)
        # if avg_f1 > best_f1:
        #     best_f1 = avg_f1
        #     count = 0 # reset count
        #     #saving best model thus far
        #     torch.save(model.state_dict(), '%s.pth'%(model.name))
        # else:
        #     count += 1
        #     if count == patience:
        #         print('Early stopping at epoch %d'%(epoch))
        #         break

        #------------- Early stopping with loss------------
        if loss_test.item() < best_loss:
            best_loss = loss_test.item()
            count = 0 # reset count
            best_epoch = epoch
            #saving best model thus far
            torch.save(model.state_dict(), '%s.pth'%(model.name))
            if best_loss < 0.0001:
                print('\n----------------------------------------\nEarly stopping at epoch %d'%(epoch))
                print('Best epoch: ', best_epoch)
                print('Best loss: ', best_loss, ' Best test F1 score: ', f1_test, ' Best test accuracy: ', acc_list_test[-1])
                break
        else:
            count += 1
            if count == patience:
                print('\n----------------------------------------\nEarly stopping at epoch %d'%(epoch))
                print('Best epoch: ', best_epoch)
                print('Best loss: ', best_loss, ' Best test F1 score: ', f1_test, ' Best test accuracy: ', acc_list_test[-1])
                break

        



In [16]:
model1 = CNN().to(device)
train_and_save_model(model1, data_loader, valid_loader, device)


Epoch [1/1000], Step [1/44], Loss: 0.5483, Accuracy: 83.33%, F1 Score: 0.80
Epoch [1/1000], Step [2/44], Loss: 0.5572, Accuracy: 83.33%, F1 Score: 0.69
Epoch [1/1000], Step [3/44], Loss: 0.6818, Accuracy: 83.33%, F1 Score: 0.65
Epoch [1/1000], Step [4/44], Loss: 0.5289, Accuracy: 83.33%, F1 Score: 0.77
Epoch [1/1000], Step [5/44], Loss: 0.7828, Accuracy: 83.33%, F1 Score: 0.62
Epoch [1/1000], Step [6/44], Loss: 0.6398, Accuracy: 83.33%, F1 Score: 0.66
Epoch [1/1000], Step [7/44], Loss: 0.8041, Accuracy: 83.33%, F1 Score: 0.61
Epoch [1/1000], Step [8/44], Loss: 0.5803, Accuracy: 83.33%, F1 Score: 0.71
Epoch [1/1000], Step [9/44], Loss: 0.6500, Accuracy: 83.33%, F1 Score: 0.66
Epoch [1/1000], Step [10/44], Loss: 0.4193, Accuracy: 83.33%, F1 Score: 0.64
0.4192916452884674
Epoch [2/1000], Step [1/44], Loss: 0.3298, Accuracy: 83.33%, F1 Score: 0.86
Epoch [2/1000], Step [2/44], Loss: 0.5101, Accuracy: 83.33%, F1 Score: 0.73
Epoch [2/1000], Step [3/44], Loss: 0.3760, Accuracy: 83.33%, F1 Scor

In [17]:
#device = torch.device('cpu')

model2 = CNN2().to(device)

train_and_save_model(model2, data_loader, valid_loader, device)


Epoch [1/1000], Step [1/44], Loss: 0.6978, Accuracy: 87.50%, F1 Score: 0.71
Epoch [1/1000], Step [2/44], Loss: 0.4584, Accuracy: 87.50%, F1 Score: 0.83
Epoch [1/1000], Step [3/44], Loss: 0.6446, Accuracy: 87.50%, F1 Score: 0.68
Epoch [1/1000], Step [4/44], Loss: 0.4771, Accuracy: 87.50%, F1 Score: 0.78
Epoch [1/1000], Step [5/44], Loss: 0.4647, Accuracy: 87.50%, F1 Score: 0.79
Epoch [1/1000], Step [6/44], Loss: 0.5236, Accuracy: 87.50%, F1 Score: 0.76
Epoch [1/1000], Step [7/44], Loss: 0.6692, Accuracy: 87.50%, F1 Score: 0.74
Epoch [1/1000], Step [8/44], Loss: 0.6129, Accuracy: 87.50%, F1 Score: 0.79
Epoch [1/1000], Step [9/44], Loss: 0.4370, Accuracy: 87.50%, F1 Score: 0.84
Epoch [1/1000], Step [10/44], Loss: 0.4274, Accuracy: 87.50%, F1 Score: 0.89
0.42743048071861267
Epoch [2/1000], Step [1/44], Loss: 0.5160, Accuracy: 87.50%, F1 Score: 0.77
Epoch [2/1000], Step [2/44], Loss: 0.2056, Accuracy: 87.50%, F1 Score: 0.94
Epoch [2/1000], Step [3/44], Loss: 0.3175, Accuracy: 87.50%, F1 Sco

In [18]:

model3 = CNN3().to(device)
train_and_save_model(model3, data_loader, valid_loader, device)

Epoch [1/1000], Step [1/44], Loss: 0.5850, Accuracy: 75.00%, F1 Score: 0.85
Epoch [1/1000], Step [2/44], Loss: 0.6231, Accuracy: 75.00%, F1 Score: 0.71
Epoch [1/1000], Step [3/44], Loss: 0.6847, Accuracy: 75.00%, F1 Score: 0.64
Epoch [1/1000], Step [4/44], Loss: 0.7283, Accuracy: 75.00%, F1 Score: 0.72
Epoch [1/1000], Step [5/44], Loss: 0.4213, Accuracy: 75.00%, F1 Score: 0.81
Epoch [1/1000], Step [6/44], Loss: 0.3902, Accuracy: 75.00%, F1 Score: 0.80
Epoch [1/1000], Step [7/44], Loss: 0.7587, Accuracy: 75.00%, F1 Score: 0.82
Epoch [1/1000], Step [8/44], Loss: 0.3336, Accuracy: 75.00%, F1 Score: 0.90
Epoch [1/1000], Step [9/44], Loss: 0.2349, Accuracy: 75.00%, F1 Score: 0.92
Epoch [1/1000], Step [10/44], Loss: 0.7178, Accuracy: 75.00%, F1 Score: 0.56
0.7177503108978271
Epoch [2/1000], Step [1/44], Loss: 0.3385, Accuracy: 100.00%, F1 Score: 0.86
Epoch [2/1000], Step [2/44], Loss: 0.3551, Accuracy: 100.00%, F1 Score: 0.86
Epoch [2/1000], Step [3/44], Loss: 0.4911, Accuracy: 100.00%, F1 S

In [27]:

model4 = CNN4().to(device)
train_and_save_model(model4, data_loader, valid_loader, device)

Epoch [1/1000], Step [1/44], Loss: 0.9792, Accuracy: 50.00%, F1 Score: 0.50
Epoch [1/1000], Step [2/44], Loss: 0.7398, Accuracy: 50.00%, F1 Score: 0.64
Epoch [1/1000], Step [3/44], Loss: 1.0003, Accuracy: 50.00%, F1 Score: 0.45
Epoch [1/1000], Step [4/44], Loss: 1.1082, Accuracy: 50.00%, F1 Score: 0.61
Epoch [1/1000], Step [5/44], Loss: 0.8663, Accuracy: 50.00%, F1 Score: 0.58
Epoch [1/1000], Step [6/44], Loss: 1.3947, Accuracy: 50.00%, F1 Score: 0.53
Epoch [1/1000], Step [7/44], Loss: 0.6802, Accuracy: 50.00%, F1 Score: 0.73
Epoch [1/1000], Step [8/44], Loss: 1.1705, Accuracy: 50.00%, F1 Score: 0.43
Epoch [1/1000], Step [9/44], Loss: 0.7596, Accuracy: 50.00%, F1 Score: 0.70
Epoch [1/1000], Step [10/44], Loss: 0.6634, Accuracy: 50.00%, F1 Score: 0.68
0.663383960723877
Epoch [2/1000], Step [1/44], Loss: 0.8791, Accuracy: 66.67%, F1 Score: 0.54
Epoch [2/1000], Step [2/44], Loss: 0.4065, Accuracy: 66.67%, F1 Score: 0.71
Epoch [2/1000], Step [3/44], Loss: 0.6760, Accuracy: 66.67%, F1 Score

In [10]:
def evaluate_model(model, valid_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        all_labels = []
        all_predictions = []
        for images, labels in valid_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
        print('Test Accuracy of the model (validation): {:.2f} %'
        .format((correct / total) * 100))
        print('F1 Score of the model (validation): {:.2f} %'
        .format((f1_score(all_labels, all_predictions, average='macro')) * 100))

def test_model(model, test_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        all_labels = []
        all_predictions = []
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
        print('Test Accuracy of the model (testing): {:.2f} %'
        .format((correct / total) * 100))
        print('F1 Score of the model (testing): {:.2f} %'
        .format((f1_score(all_labels, all_predictions, average='macro')) * 100))

In [28]:
for model in [model1, model2, model3, model4]:
    print('\nModel: ', model.name, '\n')

    evaluate_model(model, valid_loader)
    print('\n')
    test_model(model, test_loader)


Model:  CNN 



Test Accuracy of the model (validation): 91.33 %
F1 Score of the model (validation): 91.69 %


Test Accuracy of the model (testing): 90.67 %
F1 Score of the model (testing): 90.92 %

Model:  CNN2 

Test Accuracy of the model (validation): 90.00 %
F1 Score of the model (validation): 90.05 %


Test Accuracy of the model (testing): 89.67 %
F1 Score of the model (testing): 89.80 %

Model:  CNN3 

Test Accuracy of the model (validation): 87.67 %
F1 Score of the model (validation): 87.72 %


Test Accuracy of the model (testing): 90.33 %
F1 Score of the model (testing): 90.42 %

Model:  CNN4 

Test Accuracy of the model (validation): 89.00 %
F1 Score of the model (validation): 89.30 %


Test Accuracy of the model (testing): 88.67 %
F1 Score of the model (testing): 88.98 %


In [ ]:
# function to test an individual image from an external source
def test_individual_image(model, image_name, category, read_custom_path = ''):
    if (read_custom_path != ''):
        img = cv2.imread(read_custom_path, 0)
        image_name = read_custom_path
        #save the image
    else:
        img = cv2.imread("../concat_data/%s/%s" % (category, image_name), 0)
    
    #resize to 48x48 pixels
    img = cv2.resize(img, (48, 48))



    #if image 3 channel convert to 1 channel
    if len(img.shape) > 2:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    cv2.imwrite("image_mod.jpg", img)
    
    # Convert image to tensor and add batch and channel dimensions
    img_tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    
    # Get label tensor
    labels = {'focused': 0, 'happy': 1, 'neutral': 2, 'surprised': 3}
    reverse_labels = {0: 'focused', 1: 'happy', 2: 'neutral', 3: 'surprised'}
    label_tensor = torch.tensor(labels[category], dtype=torch.long)

    # Forward pass for the single image
    img_tensor = img_tensor.to(device)
    output = model(img_tensor)
    _, predicted = torch.max(output, 1)

    # Print results
    print("Predicted:", predicted.item(), "(", reverse_labels[predicted.item()], ")")
    print("Actual:", label_tensor.item(), "(", reverse_labels[label_tensor.item()], ")")
    print("Image:", image_name)
    print("Category:", category)
    print("///////")
    
    return predicted.item()

In [ ]:
test_individual_image(model2, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "happy", read_custom_path=r"../unseen-test-imgs/test_smile.PNG")


Predicted: 1 ( happy )
Actual: 1 ( happy )
Image: ../unseen-test-imgs/test_smile.PNG
Category: happy
///////


1

In [ ]:
test_individual_image(model3, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "neutral", read_custom_path=r"../unseen-test-imgs/test_neutral.PNG")

Predicted: 1 ( happy )
Actual: 2 ( neutral )
Image: ../unseen-test-imgs/test_neutral.PNG
Category: neutral
///////


1

In [ ]:
test_individual_image(model2, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "focused", read_custom_path=r"../unseen-test-imgs/test_focused.PNG")
test_individual_image(model2, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "focused", read_custom_path=r"../unseen-test-imgs/test_focused_2.PNG")
test_individual_image(model2, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "focused", read_custom_path=r"../unseen-test-imgs/test_focused_3.PNG")
test_individual_image(model2, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "focused", read_custom_path=r"../unseen-test-imgs/test_focused_4.PNG")
test_individual_image(model2, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "focused", read_custom_path=r"../unseen-test-imgs/test_focused_5.PNG")

Predicted: 0 ( focused )
Actual: 0 ( focused )
Image: ../unseen-test-imgs/test_focused.PNG
Category: focused
///////
Predicted: 0 ( focused )
Actual: 0 ( focused )
Image: ../unseen-test-imgs/test_focused_2.PNG
Category: focused
///////
Predicted: 0 ( focused )
Actual: 0 ( focused )
Image: ../unseen-test-imgs/test_focused_3.PNG
Category: focused
///////
Predicted: 0 ( focused )
Actual: 0 ( focused )
Image: ../unseen-test-imgs/test_focused_4.PNG
Category: focused
///////
Predicted: 0 ( focused )
Actual: 0 ( focused )
Image: ../unseen-test-imgs/test_focused_5.PNG
Category: focused
///////


0

In [ ]:
test_individual_image(model1, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "surprised", read_custom_path=r"../unseen-test-imgs/test_surprised.PNG")

Predicted: 3 ( surprised )
Actual: 3 ( surprised )
Image: ../unseen-test-imgs/test_surprised.PNG
Category: surprised
///////


3

In [ ]:
test_individual_image(model1, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "focused")

[ WARN:0@536.177] global loadsave.cpp:248 findDecoder imread_('../concat_data/focused/86_MMA-FACIAL-EXPRESSION-mahmoud.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
